In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from plotnine import *
%matplotlib inline

In [ ]:
df = pd.read_csv('../input/amphibians-data-set/dataset.csv',delimiter=';')
new_header = df.iloc[0]
df = df[1:]
df.columns = new_header

# EDA

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
for feature in df.columns:
    print(feature,df[feature].isnull().sum(),'of',df.shape[0],'values are missing')

We have no missing values in our dataset.

Numerical Variables: ID, SR, NR, OR. <br>
Categorical Variables: all the rest. Some of them are ordinal.

In [ ]:
numerical_variables = ['ID','SR','NR','OR']
categorical_variables = list(set(df.columns)-set(numerical_variables))

In [ ]:
category2 = ["SUR1", "SUR2", "SUR3"]
for c in category2:
    print("{} \n".format(df[c].value_counts()))

In [ ]:
a = ['1','2','10','3']
sorted(int(i) for i in a)

In [ ]:
ds = df.copy()
#ds['Motorway'] = ds['Motorway'].replace('A1', 1)
#ds['Motorway'] = ds['Motorway'].replace('S52', 2)
#ds = ds.apply(pd.to_numeric) 
#ds = ds.reindex(sorted(ds.columns), axis=1)
                

In [ ]:
from plotnine import options
options.figure_size = (6,2)
for i in categorical_variables:
    ds[i] = sorted(ds[i])
    (print(   
            ggplot(ds)
            +aes(i,fill = 'Motorway')
            +geom_bar(width=0.3)
            +ggtitle(''))
    ) 

In [ ]:
ds.columns

In [ ]:
sns.pairplot(ds,vars=ds[numerical_variables[1:]],hue='Motorway')

In [ ]:
species = ['Green frogs','Brown frogs','Common toad','Fire-bellied toad','Tree frog',\
    'Common newt','Great crested newt']
df['Species'] = df[species].astype(np.int).sum(axis=1)

In [ ]:
data_copy = df.copy()

data_copy['Motorway'] = data_copy['Motorway'].replace('A1', 1)
data_copy['Motorway'] = data_copy['Motorway'].replace('S52', 2)
data_copy = data_copy.apply(pd.to_numeric) 
data_copy = data_copy.drop(['ID'],axis=1)
plt.figure(figsize=(16, 6))
ax = sns.heatmap(data_copy.corr(),vmin=-1,vmax=1, annot=True)


In [ ]:
df.head()

In [ ]:
df=df.drop(['Green frogs', 'Brown frogs','Common toad','Fire-bellied toad','Tree frog',\
            'Common toad','Fire-bellied toad','Tree frog','Common newt',\
            'Great crested newt'], axis=1)

In [ ]:
df.head()

In [ ]:
for i in df.columns[1:-1]:
    print(df[[i,"Species"]].groupby(i, as_index = False).mean()\
        .sort_values(by="Species",ascending = False))
    print()

In [ ]:
df['Motorway'].unique()

In [ ]:
data = df.copy()

data['Motorway'] = data['Motorway'].replace('A1', 1)
data['Motorway'] = data['Motorway'].replace('S52', 2)

#ax = sns.heatmap(data, annot=True)

In [ ]:
data = data.apply(pd.to_numeric) 

In [ ]:
data = data.drop(['ID'],axis=1)

In [ ]:
plt.figure(figsize=(16, 6))
ax = sns.heatmap(data.corr(),vmin=-1,vmax=1, annot=True)

- Highly positively correlated features:<br>NR and SR, FR and UR, RR and BR.
- Highly negatively correlated features:<br>UR and VR, FR and VR. 
- Species are weakly correlated with VR, SUR1, SUR2, RR, BR.

In [ ]:
def detect_outliers(df2,features):
    outlier_indices = []
    
    for c in features:
        # 1st quartile
        Q1 = np.percentile(df2[c],25)
        # 3rd quartile
        Q3 = np.percentile(df2[c],75)
        # IQR
        IQR = Q3 - Q1
        # Outlier step
        outlier_step = IQR * 1.5

        # detect outlier and their indeces
        outlier_list_col = df2[(df2[c] < Q1 - outlier_step) | \
                               (df2[c] > Q3 + outlier_step)].index
        # store indeces
        outlier_indices.extend(outlier_list_col)
    
    outlier_indices = Counter(outlier_indices)
    multiple_outliers = list(i for i, v in outlier_indices.items() if v > 2)
    
    return multiple_outliers

In [ ]:
df.loc[detect_outliers(df,["TR","VR","UR","FR","OR","RR","BR","CR"])]